In [1]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModelForCausalLM, pipeline
from transformers.models.bert import BertForMaskedLM
from transformers.models.gpt2 import GPT2LMHeadModel

/Users/bochnak/anaconda3/envs/nlp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Download three Polish models from the Huggingface repository.
These should be regular language models, which were not fine-tuned. E.g. HerBERT and papuGaPT2 are good examples. You can also try using Bielik for that, but make sure you are using the model via Transformers API, not GUI.



In [2]:
models = {
    "HerBERT": {
        "tokenizer": AutoTokenizer.from_pretrained("allegro/herbert-base-cased"),
        "model": AutoModelForMaskedLM.from_pretrained("allegro/herbert-base-cased"),
        "pipeline": None,
    },
    "papuGaPT2": {
        "tokenizer": AutoTokenizer.from_pretrained("flax-community/papuGaPT2"),
        "model": AutoModelForCausalLM.from_pretrained("flax-community/papuGaPT2"),
        "pipeline": None,
    },
    "Bielik": {
        "tokenizer": AutoTokenizer.from_pretrained("dkleczek/bert-base-polish-cased-v1"),
        "model": AutoModelForMaskedLM.from_pretrained("dkleczek/bert-base-polish-cased-v1"),
        "pipeline": None,
    }
}

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


#### Devise a method to test if the langage model understands Polish cases.
 E.g. testing for nominal case could be expressed as "Warszawa to największe [MASK]", and the masked word should be in nominative case. Create sentences for each case.

In [3]:
for model_name, model_info in models.items():
    model = model_info["model"]

    if isinstance(model, BertForMaskedLM):
        fill_mask_pipeline = pipeline("fill-mask", model=model, tokenizer=model_info["tokenizer"])
        models[model_name]["pipeline"] = fill_mask_pipeline
        print(f"Pipeline (fill-mask) successfully assigned for model: {model_name}")

    elif isinstance(model, GPT2LMHeadModel):
        text_generation_pipeline = pipeline(
            "text-generation",
            model=model,
            tokenizer=model_info["tokenizer"],
            max_new_tokens=2,
            pad_token_id=model_info["tokenizer"].eos_token_id,
            eos_token_id=model_info["tokenizer"].eos_token_id,
            top_k=50,
            top_p=0.9
        )
        models[model_name]["pipeline"] = text_generation_pipeline
        print(f"Pipeline (text-generation) successfully assigned for model: {model_name}")

    else:
        print(f"Model {model_name} is not compatible with the intended pipelines.")


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Pipeline (fill-mask) successfully assigned for model: HerBERT
Pipeline (text-generation) successfully assigned for model: papuGaPT2
Pipeline (fill-mask) successfully assigned for model: Bielik


In [4]:
masks = {
    "HerBERT": "<mask>.",
    "Bielik": "[MASK].",
    "papuGaPT2" : ""
}

case_sentences = {
    "nominative": "Warszawa to największe {mask}",
    "genitive": "To ciężkie zadanie, potrzebuję trochę więcej {mask}",
    "dative": "Mój pies Zeus przyglądał się {mask}",
    "accusative": "Wczoraj w stawie widziałem dużą {mask}",
    "instrumental": "Wczoraj podróżowałem z moim {mask}",
    "locative": "Wczoraj po wywiadówce z moją mamą rozmawialiśmy o moich {mask}",
}

for model_name, model_info in models.items():
    if model_info["pipeline"]:
        print(f"\nTesting cases with {model_name} model:")

        mask = masks.get(model_name, "<mask>")

        for case, sentence in case_sentences.items():
            sentence_with_mask = sentence.format(mask=mask)
            print(f"\nCase: {case}")
            outputs = model_info["pipeline"](sentence_with_mask)
            if model_name in ["HerBERT", "Bielik"]:
                for output in outputs[:3]:
                    print(f"Prediction: {output['sequence']} (Score: {output['score']})")
            elif model_name == "papuGaPT2":
                for output in outputs[:3]:
                    print(f"Prediction: {output['generated_text']}")



Testing cases with HerBERT model:

Case: nominative
Prediction: Warszawa to największe miasto . (Score: 0.8103939294815063)
Prediction: Warszawa to największe lotnisko . (Score: 0.08249125629663467)
Prediction: Warszawa to największe centrum . (Score: 0.026521336287260056)

Case: genitive
Prediction: To ciężkie zadanie , potrzebuję trochę więcej czasu . (Score: 0.4029397666454315)
Prediction: To ciężkie zadanie , potrzebuję trochę więcej cierpliwości . (Score: 0.11514861136674881)
Prediction: To ciężkie zadanie , potrzebuję trochę więcej doświadczenia . (Score: 0.09145424515008926)

Case: dative
Prediction: Mój pies Zeus przyglądał się temu . (Score: 0.3961878716945648)
Prediction: Mój pies Zeus przyglądał się wszystkiemu . (Score: 0.13475088775157928)
Prediction: Mój pies Zeus przyglądał się mu . (Score: 0.05179081857204437)

Case: accusative
Prediction: Wczoraj w stawie widziałem dużą burzę . (Score: 0.1044120118021965)
Prediction: Wczoraj w stawie widziałem dużą rzekę . (Score: 0.0

#### Devise a method to test long-range relationships such as gender.
E.e. you can use two verbs with masculine and feminine gender, where one of the verbs is masked. Both verbs should have the same gender, assuming the subject is the same. Define at least 3 such sentences.



In [5]:
gender_sentences = [
    "Maria jest świetną lekarką i zawsze dokładnie słucha swoich pacjentów, zanim ich {mask}",
    "Andrzej spędził dużo czasu nad zadaniami zanim wszystkie je {mask}",
    "Agnieszka była zmęczona, ale pomimo to dalej {mask}",
]

for model_name, model_info in models.items():
    if model_info["pipeline"]:
        mask = masks.get(model_name, "<mask>")
        print(f"\nTesting gender relationships with {model_name} model:")
        for sentence in gender_sentences:
            sentence_with_mask = sentence.format(mask=mask)
            outputs = model_info["pipeline"](sentence_with_mask)
            if model_name in ["HerBERT", "Bielik"]:
                for output in outputs[:3]:  # Display top 3 predictions
                    print(f"Prediction: {output['sequence']} (Score: {output['score']})")
            elif model_name == "papuGaPT2":
                for output in outputs[:3]:  # Display top 3 predictions
                    print(f"Prediction: {output['generated_text']}")



Testing gender relationships with HerBERT model:
Prediction: Maria jest świetną lekarką i zawsze dokładnie słucha swoich pacjentów , zanim ich leczy . (Score: 0.2329351007938385)
Prediction: Maria jest świetną lekarką i zawsze dokładnie słucha swoich pacjentów , zanim ich odwiedzi . (Score: 0.1360275000333786)
Prediction: Maria jest świetną lekarką i zawsze dokładnie słucha swoich pacjentów , zanim ich postawi . (Score: 0.13440170884132385)
Prediction: Andrzej spędził dużo czasu nad zadaniami zanim wszystkie je wykonał . (Score: 0.38248470425605774)
Prediction: Andrzej spędził dużo czasu nad zadaniami zanim wszystkie je rozwiązał . (Score: 0.10732978582382202)
Prediction: Andrzej spędził dużo czasu nad zadaniami zanim wszystkie je ukończył . (Score: 0.09674196690320969)
Prediction: Agnieszka była zmęczona , ale pomimo to dalej żyła . (Score: 0.16098259389400482)
Prediction: Agnieszka była zmęczona , ale pomimo to dalej walczyła . (Score: 0.14678335189819336)
Prediction: Agnieszka była

#### Check if the model captures real-world knolwedge. For instance a sentence "[MASK] wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza."
 checks if the model "knows" the description of water. Define at least 3 such sentences.

In [6]:
knowledge_sentences = [
    "Stolicą Polski jest {mask}",
    "Najwyższą górą Polski są {mask}",
    "Najlepszy w historii piłkarz z Argentyny to {mask}",
    "Stolicą Niemiec jest {mask}",
    "Stolicą Francji jest {mask}",
    "2 wojnę światową rozpoczęli {mask}",
    "2 wojnę światową wygrali {mask}"
]

for model_name, model_info in models.items():
    if model_info["pipeline"]:
        mask = masks.get(model_name, "<mask>")
        print(f"\nTesting real-world knowledge with {model_name} model:")
        for sentence in knowledge_sentences:
            sentence_with_mask = sentence.format(mask=mask)
            outputs = model_info["pipeline"](sentence_with_mask)
            if model_name in ["HerBERT", "Bielik"]:
                for output in outputs[:3]:  # Display top 3 predictions
                    print(f"Prediction: {output['sequence']} (Score: {output['score']})")
            elif model_name == "papuGaPT2":
                for output in outputs[:3]:  # Display top 3 predictions
                    print(f"Prediction: {output['generated_text']}")



Testing real-world knowledge with HerBERT model:
Prediction: Stolicą Polski jest Warszawa . (Score: 0.609553337097168)
Prediction: Stolicą Polski jest Kraków . (Score: 0.0991559699177742)
Prediction: Stolicą Polski jest Gdańsk . (Score: 0.08018141239881516)
Prediction: Najwyższą górą Polski są Niemcy . (Score: 0.5421215891838074)
Prediction: Najwyższą górą Polski są Polacy . (Score: 0.07686690241098404)
Prediction: Najwyższą górą Polski są Austriacy . (Score: 0.040299177169799805)
Prediction: Najlepszy w historii piłkarz z Argentyny to . . (Score: 0.1165275052189827)
Prediction: Najlepszy w historii piłkarz z Argentyny to Francuz . (Score: 0.02994181029498577)
Prediction: Najlepszy w historii piłkarz z Argentyny to Polak . (Score: 0.025129610672593117)
Prediction: Stolicą Niemiec jest Berlin . (Score: 0.693077564239502)
Prediction: Stolicą Niemiec jest Monachium . (Score: 0.16482771933078766)
Prediction: Stolicą Niemiec jest Stuttgart . (Score: 0.1128314808011055)
Prediction: Stolicą 

#### Check zero-shot learning capabilites of the models.
Provide at least 5 sentences with different sentiment for the following scheme: "'Ten film to był kiler. Nie mogłem się oderwać od ekranu.' Wypowiedź ta ma jest zdecydowanie [MASK]" Try different prompts, to see if they make any difference.

In [7]:
sentiment_sentences = [
    "'Ten film był super. Nie mogłem się oderwać od ekranu.' Wypowiedź ta jest zdecydowanie {mask}",
    "'To był bardzo nudny film, ledwo dotrwałem do końca.' Wypowiedź ta jest zdecydowanie {mask}",
    "'Jestem zadowolony z tego zakupu, jakość jest świetna!' Wypowiedź ta jest zdecydowanie {mask}",
    "'Nie polecam tego produktu, zawiodłem się.' Wypowiedź ta jest zdecydowanie {mask}",
    "'Książka była ciekawa, i bardzo mnie wciągnęła.' Wypowiedź ta jest zdecydowanie {mask}",
]

for model_name, model_info in models.items():
    if model_info["pipeline"]:
        mask = masks.get(model_name, "<mask>")
        print(f"\nTesting zero-shot sentiment analysis with {model_name} model:")
        for sentence in sentiment_sentences:
            sentence_with_mask = sentence.format(mask=mask)
            outputs = model_info["pipeline"](sentence_with_mask)
            if model_name in ["HerBERT", "Bielik"]:
                for output in outputs[:3]:  # Display top 3 predictions
                    print(f"Prediction: {output['sequence']} (Score: {output['score']})")
            elif model_name == "papuGaPT2":
                for output in outputs[:3]:  # Display top 3 predictions
                    print(f"Prediction: {output['generated_text']}")


Testing zero-shot sentiment analysis with HerBERT model:
Prediction: ' Ten film był super . Nie mogłem się oderwać od ekranu . ' Wypowiedź ta jest zdecydowanie prawdziwa . (Score: 0.19030681252479553)
Prediction: ' Ten film był super . Nie mogłem się oderwać od ekranu . ' Wypowiedź ta jest zdecydowanie słuszna . (Score: 0.13067613542079926)
Prediction: ' Ten film był super . Nie mogłem się oderwać od ekranu . ' Wypowiedź ta jest zdecydowanie najlepsza . (Score: 0.12920497357845306)
Prediction: ' To był bardzo nudny film , ledwo dotrwałem do końca . ' Wypowiedź ta jest zdecydowanie prawdziwa . (Score: 0.2557086944580078)
Prediction: ' To był bardzo nudny film , ledwo dotrwałem do końca . ' Wypowiedź ta jest zdecydowanie słuszna . (Score: 0.1446705013513565)
Prediction: ' To był bardzo nudny film , ledwo dotrwałem do końca . ' Wypowiedź ta jest zdecydowanie zła . (Score: 0.08947570621967316)
Prediction: ' Jestem zadowolony z tego zakupu , jakość jest świetna ! ' Wypowiedź ta jest zdecyd

#### Answer the following questions (2 points):

1. Which of the models produced the best results?

The best results were produced by the masked language models, HerBERT and Bielik. Overall, Bielik’s responses were closer to the expected answers.

2. Was any of the models able to capture Polish grammar?

Both masked language models handled Polish grammar well, providing grammatically correct responses in each case. However, the responses from the causal language model were often incorrect.

3. Was any of the models able to capture long-distant relationships between the words?

Both masked language models captured long-distance relationships between words. However it should be clear from the context that the word the model predicts should be used in that specific gender.

4. Was any of the models able to capture world knowledge?

Partially. Both models were able to predict some world knowledge, such as country capitals and certain historical facts. However, they struggled with popular knowledge, like identifying Argentina’s most famous football player or specific geographic data.

5. Was any of the models good at doing zero-shot classification?

No, neither model was precise in zero-shot classification. The predictions in each case were similar, making them difficult to distinguish.

6. What are the most striking errors made by the models?

The most striking errors were made by the causal language model. Its responses did not fit the context of the sentences. Since the GPT-2 architecture was not designed for masked language modeling, it struggled with filling masked tokens accurately.